In [71]:
import pandas as pd
import sklearn as sk
import seaborn as sns
import json
import copy

sns.set(style="darkgrid")

# Load an example dataset with long-form data
#fmri = sns.load_dataset("fmri")

# Plot the responses for different events and regions
#plt = sns.lineplot(x="timepoint", y="signal",
#             hue="region", style="event",
#             data=fmri)


#plt.show()

## Load geolocalized points for communes

In [72]:
points_communes = pd.read_csv("PLZO_CSV_WGS84.csv",sep=";",encoding="cp1252")
print(points_communes.shape)

# Drop thurgau for now
points_communes = points_communes[points_communes.Kantonskürzel!="TG"]
print(points_communes.shape)

points_communes.head()

(4138, 9)
(3937, 9)


,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kantonskürzel,E,N,Sprache
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,8.488,47.267,de
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,8.494,47.283,de
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,8.431,47.288,de
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,8.449,47.279,de
4,Bonstetten,8906,0,Bonstetten,3,ZH,8.468,47.316,de


## Load cleaned communes population data

In [73]:

with open('../1_pop_cleaning/communes_V2_checkpoint_1553875811028.json', 'r') as cf:
    data = json.load(cf)


In [74]:
#cdata = pd.DataFrame(communes)
#cdata

reviewedCommunes = data["reviewedCommunes"]
communesToReview = data["communesToReview"]

# remember that some are reviewed and some not
for commune in reviewedCommunes:
    commune["hand_reviewed"] = True
for commune in communesToReview:
    commune["hand_reviewed"] = False

print("communes[0].keys()")
print(reviewedCommunes[0].keys())
print("communesToReview[0].keys()")
print(communesToReview[0].keys())


communes[0].keys()
dict_keys(['url', 'text', 'hab_year', 'problem_time', 'problem_unit', 'problem_start', 'name', 'gravity', 'actionsDone', 'actionsUndone', 'canton', 'firstmention', 'notes', 'problems', 'warnings', 'hand_reviewed'])
communesToReview[0].keys()
dict_keys(['url', 'text', 'hab_year', 'problem_time', 'problem_unit', 'problem_start', 'name', 'gravity', 'old_hab_year', 'canton', 'firstmention', 'problems', 'warnings', 'cutoffIndex', 'tolate_hab_year', 'mismatches', 'nbErrors', 'actionsDone', 'actionsUndone', 'hand_reviewed'])


In [75]:
columns_communes = ["name","canton","url","firstmention","hab_year","notes"]

communes = reviewedCommunes+communesToReview
communes = [
    {
        "name": commune["name"],
        "canton": commune["canton"],
        "url": commune["url"],
        "firstmention": commune["firstmention"] if "firstmention" in commune else -1,
        "hab_year": [
            {
                "year": hy["year"],
                "pop": hy["pop"],
                "unit": hy["unit"] if "unit" in hy else "undefined"
            } for hy in commune["hab_year"]
        ],
        "notes": commune["notes"] if "notes" in commune else ""
    }
    for commune in communes
]
dfcommunes = pd.DataFrame(communes)[columns_communes]
print(dfcommunes.shape)

# Drop thurgau for now
dfcommunes = dfcommunes[dfcommunes.canton!="TG"]
print(dfcommunes.shape)

dfcommunes

(2242, 6)
(2162, 6)


,name,canton,url,firstmention,hab_year,notes
27,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"[{'year': '1416', 'pop': '120', 'unit': 'Haush...","hab+foyer data!, unclear population count (wit..."
28,Lausanne (Gemeinde),VD,/Articles/002408/?language=de,280,"[{'year': '1219', 'pop': '8500', 'unit': 'Einw...","hab+foyer data!, data in html tabelle\nweird p..."
29,Lutry,VD,/Articles/002420/?language=de,908,"[{'year': '1850', 'pop': '2011', 'unit': 'Einw...","hab+foyer data!, with or without Savigny,"
30,Ballaigues,VD,/Articles/002528/?language=de,1228,"[{'year': '1416', 'pop': '10', 'unit': 'Hausha...","hab+foyer data!,"
31,Belmont-sur-Yverdon,VD,/Articles/002623/?language=de,1154,"[{'year': '1409', 'pop': '31', 'unit': 'Hausha...","hab+foyer data!,"
32,Borex,VD,/Articles/002498/?language=de,1135,"[{'year': '1429', 'pop': '14', 'unit': 'Feuers...","hab+foyer data!,"
33,Ependes (VD),VD,/Articles/002635/?language=de,1154,"[{'year': '1404', 'pop': '17', 'unit': 'Hausha...","hab+foyer data!,"
34,Lignerolle,VD,/Articles/002538/?language=de,1160,"[{'year': '1416', 'pop': '60', 'unit': 'Feuers...","hab+foyer data!,"
35,Premier,VD,/Articles/002542/?language=de,1316,"[{'year': '1396', 'pop': '15', 'unit': 'Feuers...","hab+foyer data!,"
36,Pully,VD,/Articles/002412/?language=de,994,"[{'year': '1764', 'pop': '626', 'unit': 'Einw'...","hab+foyer data!,"


In [76]:
dfcommunes.to_csv("communes.csv", sep=";")

## Reshape DataFrame to 1 line per datapoint

In [77]:
columns_communes_datapoints = ["year","pop","unit","name","canton","url","firstmention","hab_year","notes"]
communes_datapoints = []

for commune in communes:
    for hy in commune["hab_year"]:
        hy_dict = copy.deepcopy(commune)
        hy_dict["year"] =  hy["year"]
        hy_dict["pop"] =  hy["pop"]
        hy_dict["unit"] = hy["unit"] if "unit" in hy else "undefined"
        communes_datapoints.append(hy_dict)

dfcommunes_datapoints = pd.DataFrame(communes_datapoints)[columns_communes_datapoints]
dfcommunes_datapoints = dfcommunes_datapoints.drop(columns=["hab_year"])
print(dfcommunes_datapoints.shape)

# Drop thurgau for now
dfcommunes_datapoints = dfcommunes_datapoints[dfcommunes_datapoints.canton!="TG"]
print(dfcommunes_datapoints.shape)

dfcommunes_datapoints

(13648, 8)
(13157, 8)


,year,pop,unit,name,canton,url,firstmention,notes
192,1416,120,Haushalte,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
193,1764,1751,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
194,1803,2001,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
195,1850,2054,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
196,1900,3025,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
197,1930,3840,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
198,1950,3381,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
199,1980,2872,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
200,1990,3110,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
201,2000,2949,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."


In [78]:
dfcommunes_datapoints.to_csv("communes_datapoints.csv", sep=";")

## TODO: merge communes and point_communes using Hungarian algorithm
https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.optimize.linear_sum_assignment.html